In [1]:
import torch, numpy as np, torch_geometric, dgl, random
from torch_geometric.utils import k_hop_subgraph
from scipy.sparse.csgraph import shortest_path
from torch_geometric.utils import to_scipy_sparse_matrix

/home/dp1200@unt.ad.unt.edu/.venv/lib64/python3.9/site-packages/networkx/utils/backends.py:135: RuntimeWarning: networkx backend defined more than once: nx-loopback
  backends.update(_get_backends("networkx.backends"))


In [4]:
torch_geometric.__version__

'2.4.0'

In [2]:
from dgl.data.knowledge_graph import WN18Dataset

In [3]:
dataset = WN18Dataset()
graph = dataset[0]
train_mask = graph.edata['train_mask']
train_idx = torch.nonzero(train_mask, as_tuple=False).squeeze()
src, dst = graph.find_edges(train_idx)
rel = graph.edata['etype'][train_idx]

# entities: 40943
# relations: 18
# training edges: 141442
# validation edges: 5000
# testing edges: 5000
Done loading data from cached files.


In [4]:
src = src.view(1, -1)
dst = dst.view(1, -1)
edge_ind = torch.cat((src, dst), 0)

In [5]:
train_batch = []
pos_batch = []
indix = [i for i in range(len(src[0]))]
random.shuffle(indix)
for i in indix:
    if len(pos_batch) >= 3000 and rel[i] == 17:
        continue
    if len(train_batch) >= 3000 and rel[i] != 17:
        continue
    if len(train_batch) >= 3000 and len(pos_batch) >= 3000:
        break
    n1 = src[0][i]
    n2 = dst[0][i]
    subset, sg, new_targets, mask = k_hop_subgraph((n1, n2), 2, edge_ind, relabel_nodes=True)
    edge_atts = torch.nn.functional.one_hot(rel[mask], num_classes=36)
    #d_n1 = shortest_path(to_scipy_sparse_matrix(sg, num_nodes=len(subset)), directed=False, indices=new_targets[0])
    #d_n2 = shortest_path(to_scipy_sparse_matrix(sg, num_nodes=len(subset)), directed=False, indices=new_targets[1])
    labels = torch.zeros(len(subset), dtype=torch.int64)
    #for j in range(len(subset)):
        #labels[j] = 1 + torch.min(torch.tensor(d_n1[j]), torch.tensor(d_n2[j])) + (d_n1[j]+d_n2[j])// 2 *((d_n1[j]+d_n2[j]) // 2 + (d_n1[j]+d_n2[j]) % 2 - 1)
    labels[new_targets[0]] = 1
    labels[new_targets[1]] = 1
    labels = torch.nn.functional.one_hot(labels, num_classes=2)
    sg = torch.sparse_coo_tensor(sg, edge_atts, size=(len(subset), len(subset), 36)).coalesce().float()
    inds = torch.tensor([[new_targets[0], new_targets[1]], [new_targets[1], new_targets[0]] ]).transpose(1, 0)
    #del_tens = torch.sparse_coo_tensor(inds, torch.nn.functional.one_hot(torch.tensor([rel[i], (rel[i]+18)%36]), num_classes=36), size=sg.shape).float().coalesce()
    #sg -= del_tens
    #k = sg.to_dense()
    #print(k[new_targets[0]][new_targets[1]])
    #print(k[new_targets[1]][new_targets[0]])
    if rel[i] == 17:
        pos_batch.append((sg, labels, 1))
    else:
        train_batch.append((sg, labels, 0))
    if (len(train_batch)+len(pos_batch))%1000 == 0:
        print("Added 1000 graphs")
    #if len(train_batch) >= 5000:
     #   break
train_batch.extend(pos_batch)
random.shuffle(train_batch)


Added 1000 graphs
Added 1000 graphs
Added 1000 graphs
Added 1000 graphs
Added 1000 graphs
Added 1000 graphs


In [6]:
from torch_geometric.nn import GATConv, aggr, GCNConv
device = torch.device("cuda")

In [7]:
class AMDGCNN(torch.nn.Module):
    def __init__(self, dim_in):
        super().__init__()
        self.gcn1 = GATConv(dim_in, 32, edge_dim=36)
        self.gcn2 = GATConv(32, 32, edge_dim=36)
        self.gcn3 = GATConv(32, 32, edge_dim=36)
        self.gcn4 = GATConv(32, 1, edge_dim=36)
        self.global_pool = aggr.SortAggregation(k=30)
        self.conv1 = torch.nn.Conv1d(1, 16, 97, 97)
        self.conv2 = torch.nn.Conv1d(16, 32, 5, 1)
        self.maxpool = torch.nn.MaxPool1d(2, 2)
        self.linear1 = torch.nn.Linear(352, 128)
        self.linear2 = torch.nn.Linear(128, 1)
        self.dropout = torch.nn.Dropout(0.5)
    def forward(self, mat, edge_index):
        h1 = self.gcn1(mat, edge_index).tanh()
        h2 = self.gcn2(h1, edge_index).tanh()
        h3 = self.gcn3(h2, edge_index).tanh()
        h4 = self.gcn4(h3, edge_index).tanh()
        h = torch.cat([h1, h2, h3, h4], dim=-1)
        h = self.global_pool(h)
        h = h.view(h.size(0), 1, h.size(-1))
        h = self.conv1(h).relu()
        h = self.maxpool(h)
        h = self.conv2(h).relu()
        h = h.view(h.size(0), -1)
        h = self.linear1(h).relu()
        h = self.dropout(h)
        h = self.linear2(h).sigmoid()
        return h

In [8]:
model = DGCNN(2)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.00055)
criterion = torch.nn.BCELoss()
model.train(True)
model.to(device)

DGCNN(
  (gcn1): GATConv(2, 32, heads=1)
  (gcn2): GATConv(32, 32, heads=1)
  (gcn3): GATConv(32, 32, heads=1)
  (gcn4): GATConv(32, 1, heads=1)
  (global_pool): SortAggregation(k=30)
  (conv1): Conv1d(1, 16, kernel_size=(97,), stride=(97,))
  (conv2): Conv1d(16, 32, kernel_size=(5,), stride=(1,))
  (maxpool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=352, out_features=128, bias=True)
  (linear2): Linear(in_features=128, out_features=1, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)

In [9]:
def train_one_epoch():
    torch.enable_grad()
    l = 0
    p = 0
    for i in train_batch:
        #label = torch.nn.functional.one_hot(i[2], num_classes=36).view(1, -1).float()
        label = torch.tensor([i[2]]).float()
        g = i[0].coalesce()
        x = i[1].float()
        optimizer.zero_grad()
        out = model(x.to(device), g.to(device))
        loss = criterion(out.view(1), label.to(device))
        loss.backward()
        optimizer.step()
        p += 1
        l += loss.item()
        if p % 100 == 0:
            print(f"Loss: {l}")
            l = 0

In [23]:
for i in range(2):
    train_one_epoch()

Loss: 64.56587055474662
Loss: 62.75802912350272
Loss: 65.18894346752379
Loss: 66.01825724542141
Loss: 66.31461307166296
Loss: 62.93911770501293
Loss: 63.795827060938095
Loss: 66.53234088420868
Loss: 62.416791677475274
Loss: 64.7587914931355
Loss: 64.34812307366239
Loss: 64.93008019030094
Loss: 47.58458555772082
Loss: 51.99266542969796
Loss: 63.91779428730087
Loss: 62.360494412481785
Loss: 65.43248200535595
Loss: 62.25509777665138
Loss: 65.01172848045826
Loss: 62.96679536469651
Loss: 63.59876887500386
Loss: 63.422203273422724
Loss: 66.89356795772625
Loss: 66.85338929565923
Loss: 64.78973880410194
Loss: 65.67738893661742
Loss: 64.53751134681715
Loss: 66.8486176431179
Loss: 67.24180131405592
Loss: 63.01138451699406
Loss: 64.51765461392051
Loss: 63.44346302748108
Loss: 63.92288612439183
Loss: 63.32926355367056
Loss: 67.31593418121338
Loss: 64.466813624884
Loss: 62.44070717693201
Loss: 66.7074725238042
Loss: 65.25537287164568
Loss: 66.24481085588195
Loss: 64.24051554186985
Loss: 70.76131704

In [11]:
test_mask = graph.edata['test_mask']
test_idx = torch.nonzero(test_mask, as_tuple=False).squeeze()
src, dst = graph.find_edges(test_idx)
rel = graph.edata['etype'][test_idx]
src = src.view(1, -1)
dst = dst.view(1, -1)
edge_ind = torch.cat((src, dst), 0)
test_batch = []
pos_t_batch = []
indix = [i for i in range(len(src[0]))]
random.shuffle(indix)
for i in indix:
    if len(pos_t_batch) >= 500 and rel[i] == 17:
        continue
    if len(test_batch) >= 500 and rel[i] != 17:
        continue
    if len(test_batch) >= 500 and len(pos_t_batch) >= 500:
        break
    n1 = src[0][i]
    n2 = dst[0][i]
    subset, sg, new_targets, mask = k_hop_subgraph((n1, n2), 2, edge_ind, relabel_nodes=True)
    edge_atts = torch.nn.functional.one_hot(rel[mask], num_classes=36)
    #d_n1 = shortest_path(to_scipy_sparse_matrix(sg, num_nodes=len(subset)), directed=False, indices=new_targets[0])
    #d_n2 = shortest_path(to_scipy_sparse_matrix(sg, num_nodes=len(subset)), directed=False, indices=new_targets[1])
    labels = torch.zeros(len(subset), dtype=torch.int64)
    #for j in range(len(subset)):
     #   labels[j] = 1 + torch.min(torch.tensor(d_n1[j]), torch.tensor(d_n2[j])) + (d_n1[j]+d_n2[j])// 2 *((d_n1[j]+d_n2[j]) // 2 + (d_n1[j]+d_n2[j]) % 2 - 1)
    labels[new_targets[0]] = 1
    labels[new_targets[1]] = 1
    labels = torch.nn.functional.one_hot(labels, num_classes=2)
    sg = torch.sparse_coo_tensor(sg, edge_atts, size=(len(subset), len(subset), 36)).coalesce().float()
    inds = torch.tensor([[new_targets[0], new_targets[1]], [new_targets[1], new_targets[0]] ]).transpose(1, 0)
    #del_tens = torch.sparse_coo_tensor(inds, torch.nn.functional.one_hot(torch.tensor([rel[i], (rel[i]+18)%36]), num_classes=36), size=sg.shape).float().coalesce()
    #sg -= del_tens
    #k = sg.to_dense()
    #print(k[new_targets[0]][new_targets[1]])
    #print(k[new_targets[1]][new_targets[0]])
    if rel[i] == 17:
        pos_t_batch.append((sg, labels, 1))
    else:
        test_batch.append((sg, labels, 0))
test_batch.extend(pos_t_batch)


In [24]:
torch.no_grad()
labels = []
preds = []
for i in test_batch:
        labels.append(i[2])
        g = i[0].coalesce()
        x = i[1].float()
        out = model(x.to(device), g.to(device))
        preds.append(out.detach())

In [25]:
from torcheval.metrics import MulticlassAUROC, BinaryAUROC
metric = BinaryAUROC()
metric.update(torch.stack(preds).view(len(test_batch)).to(device), torch.tensor(labels).to(device))
metric.compute()

tensor(0.7689, dtype=torch.float64)

In [26]:
from sklearn.metrics import roc_curve, auc

In [27]:
fpr, tpr, thr = roc_curve((np.array(torch.tensor(labels).cpu())).astype(np.int64), np.array(torch.tensor(preds).cpu()).astype(np.float64), pos_label=1)
auc(fpr, tpr)

0.768864

In [16]:
preds

[tensor([[2.0983e-08]], device='cuda:0'),
 tensor([[2.7300e-05]], device='cuda:0'),
 tensor([[5.2731e-05]], device='cuda:0'),
 tensor([[3.7434e-16]], device='cuda:0'),
 tensor([[1.1657e-12]], device='cuda:0'),
 tensor([[8.5308e-05]], device='cuda:0'),
 tensor([[0.0004]], device='cuda:0'),
 tensor([[0.0013]], device='cuda:0'),
 tensor([[7.6689e-09]], device='cuda:0'),
 tensor([[0.6923]], device='cuda:0'),
 tensor([[0.8015]], device='cuda:0'),
 tensor([[0.2633]], device='cuda:0'),
 tensor([[1.3697e-05]], device='cuda:0'),
 tensor([[1.5092e-09]], device='cuda:0'),
 tensor([[1.4789e-05]], device='cuda:0'),
 tensor([[0.0003]], device='cuda:0'),
 tensor([[2.0109e-11]], device='cuda:0'),
 tensor([[5.5963e-17]], device='cuda:0'),
 tensor([[0.0001]], device='cuda:0'),
 tensor([[8.6636e-06]], device='cuda:0'),
 tensor([[0.0011]], device='cuda:0'),
 tensor([[1.3444e-06]], device='cuda:0'),
 tensor([[8.9806e-06]], device='cuda:0'),
 tensor([[0.8927]], device='cuda:0'),
 tensor([[4.1742e-14]], devi

In [17]:
labels

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
